In [3]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
data = pd.read_csv("C:/Users/Administrateur/OneDrive/Bureau/ESCPTerm2/NLP/Cours1/assignmentspamdetector/spam.csv",
                   encoding='latin-1', usecols=["v1", "v2"])

data = data.rename(columns={"v1": "label", "v2": "text"})
data.label.value_counts()
data['label_num'] = data.label.map({'ham': 0, 'spam': 1})
print(data)
# so now, after converting label into numerical values, ie label_num column, we are
# working with this newly created column (it becomes our y vector, text => X vector)

     label                                               text  label_num
0      ham  Go until jurong point, crazy.. Available only ...          0
1      ham                      Ok lar... Joking wif u oni...          0
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...          1
3      ham  U dun say so early hor... U c already then say...          0
4      ham  Nah I don't think he goes to usf, he lives aro...          0
...    ...                                                ...        ...
5567  spam  This is the 2nd time we have tried 2 contact u...          1
5568   ham              Will Ì_ b going to esplanade fr home?          0
5569   ham  Pity, * was in mood for that. So...any other s...          0
5570   ham  The guy did some bitching but I acted like i'd...          0
5571   ham                         Rofl. Its true to its name          0

[5572 rows x 3 columns]


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data["text"], data["label_num"], test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

from sklearn.feature_extraction.text import CountVectorizer

# Scikit-learn’s CountVectorizer is used to
# convert a collection of text documents to a vector of term/token counts.
# list of text documents

(4457,)
(1115,)
(4457,)
(1115,)


In [7]:
vect = CountVectorizer()
# we are applying CountVectorizer on our spam dataset

vect.fit(X_train)
# tokenize, extract tokens and build vocab, dictionary of all the tokens

print("Vocabulary size: {}".format(len(vect.vocabulary_)))
# print("Vocabulary content:\n {}".format(vect.vocabulary_))
X_train_df = vect.transform(X_train)
# encode your data, represent your text as count vector(s)

X_train_df[:3].nonzero()
prediction = dict()

Vocabulary size: 7735


In [6]:
# second method of vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
# create a vector transform
vc_tf_idf = TfidfVectorizer()
# tokenize and build vocab
vc_tf_idf.fit(data.text)
print(vc_tf_idf.vocabulary_)
print(vc_tf_idf.idf_)
# encode document
vector = vc_tf_idf.transform(data.text)
print("the dimension of your vector")
print(vector.shape)
print("the encoded vector")
print(vector.toarray())

{'go': 3550, 'until': 8030, 'jurong': 4350, 'point': 5920, 'crazy': 2327, 'available': 1303, 'only': 5537, 'in': 4087, 'bugis': 1751, 'great': 3634, 'world': 8489, 'la': 4476, 'buffet': 1749, 'cine': 2048, 'there': 7645, 'got': 3594, 'amore': 1069, 'wat': 8267, 'ok': 5504, 'lar': 4512, 'joking': 4318, 'wif': 8392, 'oni': 5533, 'free': 3358, 'entry': 2949, 'wkly': 8447, 'comp': 2165, 'to': 7756, 'win': 8405, 'fa': 3087, 'cup': 2386, 'final': 3207, 'tkts': 7743, '21st': 411, 'may': 4930, '2005': 402, 'text': 7595, '87121': 784, 'receive': 6297, 'question': 6190, 'std': 7230, 'txt': 7933, 'rate': 6242, 'apply': 1156, '08452810075over18': 77, 'dun': 2802, 'say': 6633, 'so': 7024, 'early': 2823, 'hor': 3927, 'already': 1042, 'then': 7640, 'nah': 5238, 'don': 2712, 'think': 7660, 'he': 3781, 'goes': 3558, 'usf': 8075, 'lives': 4665, 'around': 1207, 'here': 3831, 'though': 7680, 'freemsg': 3365, 'hey': 3841, 'darling': 2443, 'it': 4218, 'been': 1460, 'week': 8313, 'now': 5420, 'and': 1084, 'n

In [8]:
# choosing the best features/model, here I tested all the model in order to fin the best one
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

# creating a list of models
models = [LogisticRegression(random_state=0), SVC(kernel='rbf'), SVC(kernel='linear'), MultinomialNB(), RandomForestClassifier(n_estimators=100)]

# testing all the models, to find the one with the highest accuracy
for i in models:
    i.fit(X_train_df, y_train)  # X_train_df: count vectors
    X_test_df = vect.transform(X_test)
    prediction[i] = i.predict(X_test_df)
    print(str(i))
    # assessing the model
    from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

    print("accuracy:", accuracy_score(y_test, prediction[i]))
    conf_mat = confusion_matrix(y_test, prediction[i])
    print("confusion matrix", conf_mat, sep='\n')
    conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]

    print("normalized confusion matrix",conf_mat_normalized, " ", sep='\n')
    print()
# we found out that the model with the highest accuracy is MultinomialNB()

LogisticRegression(random_state=0)
accuracy: 0.97847533632287
confusion matrix
[[965   0]
 [ 24 126]]
normalized confusion matrix
[[1.   0.  ]
 [0.16 0.84]]
 

SVC()
accuracy: 0.979372197309417
confusion matrix
[[965   0]
 [ 23 127]]
normalized confusion matrix
[[1.         0.        ]
 [0.15333333 0.84666667]]
 

SVC(kernel='linear')
accuracy: 0.979372197309417
confusion matrix
[[961   4]
 [ 19 131]]
normalized confusion matrix
[[0.99585492 0.00414508]
 [0.12666667 0.87333333]]
 

MultinomialNB()
accuracy: 0.9838565022421525
confusion matrix
[[963   2]
 [ 16 134]]
normalized confusion matrix
[[0.99792746 0.00207254]
 [0.10666667 0.89333333]]
 

RandomForestClassifier()
accuracy: 0.97847533632287
confusion matrix
[[965   0]
 [ 24 126]]
normalized confusion matrix
[[1.   0.  ]
 [0.16 0.84]]
 



In [9]:
# classification model MultinomialNB()
print("MULTINOMIAL REGRESSION")
model = MultinomialNB()
model.fit(X_train_df, y_train)  # X_train_df: count vectors
X_test_df = vect.transform(X_test)
prediction["Multinomial"] = model.predict(X_test_df)

# assessing the model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("accuracy:", accuracy_score(y_test, prediction["Multinomial"]))
print(classification_report(y_test, prediction["Multinomial"]))
conf_mat = confusion_matrix(y_test, prediction['Multinomial'])
print("confusion matrix", conf_mat, sep='\n')
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]

print("normalized confusion matrix",conf_mat_normalized, sep='\n')

print("train score:", model.score(X_train_df, y_train), sep='\n')
print("test score:", model.score(X_test_df, y_test), " ", sep='\n')

MULTINOMIAL REGRESSION
accuracy: 0.9838565022421525
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1       0.99      0.89      0.94       150

    accuracy                           0.98      1115
   macro avg       0.98      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115

confusion matrix
[[963   2]
 [ 16 134]]
normalized confusion matrix
[[0.99792746 0.00207254]
 [0.10666667 0.89333333]]
train score:
0.9943908458604442
test score:
0.9838565022421525
 


In [10]:
# classification model LogisticRegression()
print("LOGISTIC REGRESSION")
model = LogisticRegression()
model.fit(X_train_df, y_train)  # X_train_df: count vectors
X_test_df = vect.transform(X_test)
prediction["Logistic"] = model.predict(X_test_df)

# assessing the model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("accuracy:", accuracy_score(y_test, prediction["Logistic"]))
print(classification_report(y_test, prediction["Logistic"]))
conf_mat = confusion_matrix(y_test, prediction['Logistic'])
print("confusion matrix", conf_mat, sep='\n')
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]

print("normalized confusion matrix",conf_mat_normalized, sep='\n')

print("train score:", model.score(X_train_df, y_train), sep='\n')
print("test score:", model.score(X_test_df, y_test), " ", sep='\n')


LOGISTIC REGRESSION
accuracy: 0.97847533632287
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1       1.00      0.84      0.91       150

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115

confusion matrix
[[965   0]
 [ 24 126]]
normalized confusion matrix
[[1.   0.  ]
 [0.16 0.84]]
train score:
0.9984294368409243
test score:
0.97847533632287
 
